<a href="https://colab.research.google.com/github/GergelyTuri/els_project/blob/master/notebooks/videoconversion_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The script converts `.ffii` files to avi or mp4.
By: Aliza Hacking - 2023 summer

It is also added to the ELS repo
* 11/7/23 minor changes to the script - GT

In [ ]:
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import struct
import subprocess
import os
import time

In [ ]:
video_loc = 'path/to/video/folder/'

In [ ]:
folderpath = video_loc

exten = '.ffii'
nbFiles = 0

start_time = time.time()

for root, directories, filenames in os.walk(folderpath):
    for filename in filenames:
        if filename.lower().endswith(exten):
            filepath = os.path.join(root, filename)
            print(f'Converting {filepath} ...')

            output_filepath = filepath[:-5] + '.avi'

            # Try to open teh file safely
            try:
                with open(filepath, 'rb',) as f:
                    m = f.read(8)
                    height, width = struct.unpack(">2I", m)
                    rate = "15"

                    cmdstr = ['ffmpeg', '-y', '-r', rate,
                            '-f', 'rawvideo',
                            '-pix_fmt', 'gray',
                            '-s', f"{width}x{height}",
                            '-i', '-',
                            output_filepath]

                    p = subprocess.Popen(cmdstr, stdin=subprocess.PIPE)

                    while True:
                        img = f.read(width*height)
                        if not img:
                            break
                        p.stdin.write(img)
                        m = f.read(8)
                        if not m:
                            break
                        height, width = struct.unpack(">2I", m)
                    p.stdin.close()
                    p.wait() # Wait for FFmpeg to finish

                    print(f'Saved in {output_filepath}')

                    nbFiles += 1
            except Exception as e:
                print(f"An error occurred: {e}")

print(f'Conversion of {nbFiles} files over!')
print(f"--- {time.time() - start_time:.2f} seconds elapsed ---")